# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686552


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:16,  2.73s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:53,  1.99s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:37,  1.44s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:26,  1.06s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  3.86it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  4.28it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  4.67it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  4.32it/s]

not-logged-in-0ed3d164fd32cee0b7f7    0.000286
CThomas                               0.001737
ElisabethB                            0.000257
not-logged-in-db23dc032fc427a4e4d9    0.000242
not-logged-in-4c2ff289142efbb5f0fb    0.000263
not-logged-in-9f7fd9ce240826b2ff22    0.000476
not-logged-in-2159616075e1803d6325    0.000646
kjkavene                              0.000288
jmfranci                              0.000792
not-logged-in-9e1ea3efe20c6c95e390    0.002757
not-logged-in-c08c0c43bd0e8eef780a    0.001045
not-logged-in-96a0fd5d39dcc16f9673    0.000916
lsautter                              0.000281
FelipedaFonseca                       0.000504
not-logged-in-09330e0b15263888ee9d    0.001626
pangeli5                              0.000238
fruitshake123                         0.001834
acapirala                             0.000365
not-logged-in-fcfcbc88feb1aa0ed0a7    0.000465
Lavadude                              0.034332
equidad1                              0.000346
GoatyGoats   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())